In [1]:
from pathlib import Path


PROJECT_ROOT_DIR = str(Path.cwd().parent.parent)
PROJECT_ROOT_DIR

'/root/llm_graph_embedder'

# Intro

# Loading graph dataset

In [2]:
from ogb.nodeproppred import NodePropPredDataset

dataset = NodePropPredDataset(name="ogbn-arxiv")
graph, labels = dataset[0]

/root/llm_graph_embedder/.venv/lib/python3.10/site-packages/ogb/nodeproppred/dataset.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dict = torch.load(pre_proces

# Loading nodes textual features

OGB provides a file with all text features related to a papar (title and abstract): https://ogb.stanford.edu/docs/nodeprop/#ogbn-arxiv

In [3]:
# Download file from https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv.gz and puts into PROJECT_ROOT_DIR
import requests
import os


url = "https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv.gz"
text_features_filename = Path(PROJECT_ROOT_DIR) / "titleabs.tsv.gz"

if not os.path.isfile(text_features_filename):
    response = requests.get(url)

    if response.status_code == 200:
        with open(text_features_filename, "wb") as f:
            f.write(response.content)
    else:
        print("Failed to download file.")
else:
    print("File already downloaded.")

File already downloaded.


In [4]:
import gzip
import shutil


with gzip.open(text_features_filename, "rb") as f_in:
    with open(str(text_features_filename).replace(".gz", ""), "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [5]:
node_idx_2_paper_id_f_name = "./dataset/ogbn_arxiv/mapping/nodeidx2paperid.csv"

if not os.path.isfile(node_idx_2_paper_id_f_name):
    with gzip.open(node_idx_2_paper_id_f_name + ".gz", "rb") as f_in:
        with open(node_idx_2_paper_id_f_name, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

In [6]:
import pandas as pd

text_features = pd.read_csv(
    "../titleabs.tsv",
    sep="\t",
    header=None,
    skiprows=1,
    names=["paper_id", "title", "abstract"],
    index_col="paper_id",
)

text_features = text_features.dropna()
text_features.index = text_features.index.map(int)
text_features

,title,abstract
paper_id,,
549074,a novel methodology for thermal analysis a 3 d...,The semiconductor industry is reaching a fasci...
630234,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
803423,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
1102481,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...
1532644,machine learner for automated reasoning 0 4 an...,Machine Learner for Automated Reasoning (MaLAR...
...,...,...
3012554562,federated visual classification with real worl...,Federated Learning enables visual models to be...
3012555423,kernel quantization for efficient network comp...,This paper presents a novel network compressio...
3012556759,generating electronic health records with mult...,Sharing electronic health records (EHRs) on a ...


In [7]:
node_idx_to_paper_id = pd.read_csv(
    node_idx_2_paper_id_f_name,
    index_col=0,
    names=["node_idx", "paper_id"],
    header=None,
    skiprows=1,
)
node_idx_to_paper_id

,paper_id
node_idx,
0,9657784
1,39886162
2,116214155
3,121432379
4,231147053
...,...
169338,3011696425
169339,3011708313
169340,3011798063


In [8]:
node_text_features = pd.merge(
    node_idx_to_paper_id,
    text_features,
    left_on="paper_id",
    right_on=text_features.index,
)
node_text_features.index.names = ["node_idx"]

In [9]:
node_text_features["publishing_year"] = graph["node_year"]
node_text_features

,paper_id,title,abstract,publishing_year
node_idx,,,,
0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",2013
1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,2015
2,116214155,on the absence of the rip in real world applic...,The purpose of this paper is twofold. The firs...,2014
3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,2014
4,231147053,analysis of asymptotically optimal sampling ba...,Over the last 20 years significant effort has ...,2014
...,...,...,...,...
169338,3011696425,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,2020
169339,3011708313,interpretable mtl from heterogeneous domains u...,Multi-task learning (MTL) aims at improving th...,2020
169340,3011798063,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",2020


In [10]:
node_text_features["label"] = labels
node_text_features

,paper_id,title,abstract,publishing_year,label
node_idx,,,,,
0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",2013,4
1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,2015,5
2,116214155,on the absence of the rip in real world applic...,The purpose of this paper is twofold. The firs...,2014,28
3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,2014,8
4,231147053,analysis of asymptotically optimal sampling ba...,Over the last 20 years significant effort has ...,2014,27
...,...,...,...,...,...
169338,3011696425,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,2020,4
169339,3011708313,interpretable mtl from heterogeneous domains u...,Multi-task learning (MTL) aims at improving th...,2020,24
169340,3011798063,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",2020,10


In [11]:
# Select all rows in node_text_features where publishing_year is 2018
node_text_features[node_text_features["publishing_year"] == 2018]

,paper_id,title,abstract,publishing_year,label
node_idx,,,,,
349,2582667927,flexonc joint cooperative forwarding and netwo...,"In recent years, network coding has emerged as...",2018,8
357,2591735613,statistical anomaly detection via composite hy...,"Under Markovian assumptions, we leverage a cen...",2018,19
366,2604507718,sce a manifold regularized set covering method...,Cluster analysis plays a very important role i...,2018,16
382,2611510109,understanding people flow in transportation hubs,"In this paper, we aim to monitor the flow of p...",2018,16
393,2621813772,the temporal event graph,Temporal networks are increasingly being used ...,2018,26
...,...,...,...,...,...
169155,2965531707,augmented reality based feedback for technicia...,Interventional C-arm imaging is crucial to per...,2018,16
169176,2971687264,privado practical and secure dnn inference wit...,Cloud providers are extending support for trus...,2018,4
169185,2972582134,collective learning from diverse datasets for ...,Entity typing (ET) is the problem of assigning...,2018,30


# Loading graph instructions

In [12]:
from datasets import load_dataset


train_instructions = pd.read_json(
    "hf://datasets/Jiabin99/Arxiv-PubMed-mix-NC-LP/arxiv_pub_node_st_cot_link_mix.json"
)
train_instructions = train_instructions[
    train_instructions["id"].str.startswith("arxiv_train")
]
train_instructions["node_idx"] = train_instructions["id"].apply(
    lambda x: x.split("_")[-1]
)
train_instructions.index = train_instructions["node_idx"]
train_instructions.index = train_instructions.index.map(int)

In [13]:
from datasets import load_dataset

test_instructions = load_dataset(
    "Jiabin99/GraphGPT-eval-instruction",
    split="test",
    data_files={"test": "arxiv_test_instruct_std.json"},
)
test_instructions = pd.DataFrame(test_instructions)
test_instructions["node_idx"] = test_instructions["id"].apply(
    lambda x: x.split("_")[-1]
)
test_instructions.index = test_instructions["node_idx"]
test_instructions.index = test_instructions.index.map(int)

# Merging text features and instructions

In [14]:
# Create a new dataframe that makes an inner join between `node_text_features` and `train_instructions` on the indexes of both dataframes
train_data = pd.merge(
    node_text_features, train_instructions, left_index=True, right_index=True
)
train_data

,paper_id,title,abstract,publishing_year,label,id,graph,conversations,node_idx
node_idx,,,,,,,,,
0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",2013,4,arxiv_train_0,"{'node_idx': 0, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",0
1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,2015,5,arxiv_train_1,"{'node_idx': 1, 'edge_index': [[1, 2, 3, 4, 0,...","[{'from': 'human', 'value': 'Given a citation ...",1
2,116214155,on the absence of the rip in real world applic...,The purpose of this paper is twofold. The firs...,2014,28,arxiv_train_2,"{'node_idx': 2, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",2
3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,2014,8,arxiv_train_3,"{'node_idx': 3, 'edge_index': [[1, 2, 0, 3, 4,...","[{'from': 'human', 'value': 'Given a citation ...",3
4,231147053,analysis of asymptotically optimal sampling ba...,Over the last 20 years significant effort has ...,2014,27,arxiv_train_4,"{'node_idx': 4, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",4
...,...,...,...,...,...,...,...,...,...
169113,2953374028,network error correction for unit delay memory...,A single source network is said to be memory-f...,2009,28,arxiv_train_169113,"{'node_idx': 169113, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169113
169115,2953380034,subgraph sparsification and nearly optimal ult...,We consider a variation of the spectral sparsi...,2009,39,arxiv_train_169115,"{'node_idx': 169115, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169115
169145,2963574152,hybrid spatio temporal artificial noise design...,This paper investigates artificial noise injec...,2017,28,arxiv_train_169145,"{'node_idx': 169145, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169145


In [15]:
min(train_data["publishing_year"]), max(train_data["publishing_year"])

(1986, 2017)

In [16]:
# Create a new dataframe that makes an inner join between `node_text_features` and `test_instructions` on the indexes of both dataframes
test_data = pd.merge(
    node_text_features, test_instructions, left_index=True, right_index=True
)
test_data

,paper_id,title,abstract,publishing_year,label,id,graph,conversations,node_idx
node_idx,,,,,,,,,
346,2580068393,factored contextual policy search with bayesia...,Scarce data is a major challenge to scaling ro...,2019,24,arxiv_test_346,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",346
398,2625226699,simultaneous merging multiple grid maps using ...,Mapping in the GPS-denied environment is an im...,2019,10,arxiv_test_398,"{'edge_index': [[1, 2, 3, 4, 0, 5, 6, 7, 8, 9,...","[{'from': 'human', 'value': 'Given a citation ...",398
451,2763049606,reconstruction of hidden representation for ro...,This article aims to develop a new and robust ...,2019,24,arxiv_test_451,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",451
480,2775899375,a look at the time delays in cvss vulnerabilit...,Abstract This empirical paper examines the t...,2019,4,arxiv_test_480,"{'edge_index': [[1, 2, 3, 4, 5, 6, 0, 7, 8, 9,...","[{'from': 'human', 'value': 'Given a citation ...",480
488,2782758897,common tangents of two disjoint polygons in li...,We provide a remarkably simple algorithm to co...,2019,20,arxiv_test_488,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",488
...,...,...,...,...,...,...,...,...,...
169338,3011696425,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,2020,4,arxiv_test_169338,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169338
169339,3011708313,interpretable mtl from heterogeneous domains u...,Multi-task learning (MTL) aims at improving th...,2020,24,arxiv_test_169339,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169339
169340,3011798063,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",2020,10,arxiv_test_169340,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169340


In [17]:
min(test_data["publishing_year"]), max(test_data["publishing_year"])

(2019, 2020)

# Generating full set of instructions

In [18]:
def replace_graph_in_conversations(conversation, graph_data):
    parsed_human_message = conversation[0]["value"]
    parsed_human_message = parsed_human_message.replace("<graph>", str(graph_data))

    return [{"from": "human", "value": parsed_human_message}, conversation[1]]

In [19]:
train_data["formatted_conversations"] = train_data.apply(
    lambda row: replace_graph_in_conversations(row["conversations"], row["graph"]),
    axis=1,
)

In [20]:
test_data["formatted_conversations"] = test_data.apply(
    lambda row: replace_graph_in_conversations(row["conversations"], row["graph"]),
    axis=1,
)

In [21]:
test_data

,paper_id,title,abstract,publishing_year,label,id,graph,conversations,node_idx,formatted_conversations
node_idx,,,,,,,,,,
346,2580068393,factored contextual policy search with bayesia...,Scarce data is a major challenge to scaling ro...,2019,24,arxiv_test_346,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",346,"[{'from': 'human', 'value': 'Given a citation ..."
398,2625226699,simultaneous merging multiple grid maps using ...,Mapping in the GPS-denied environment is an im...,2019,10,arxiv_test_398,"{'edge_index': [[1, 2, 3, 4, 0, 5, 6, 7, 8, 9,...","[{'from': 'human', 'value': 'Given a citation ...",398,"[{'from': 'human', 'value': 'Given a citation ..."
451,2763049606,reconstruction of hidden representation for ro...,This article aims to develop a new and robust ...,2019,24,arxiv_test_451,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",451,"[{'from': 'human', 'value': 'Given a citation ..."
480,2775899375,a look at the time delays in cvss vulnerabilit...,Abstract This empirical paper examines the t...,2019,4,arxiv_test_480,"{'edge_index': [[1, 2, 3, 4, 5, 6, 0, 7, 8, 9,...","[{'from': 'human', 'value': 'Given a citation ...",480,"[{'from': 'human', 'value': 'Given a citation ..."
488,2782758897,common tangents of two disjoint polygons in li...,We provide a remarkably simple algorithm to co...,2019,20,arxiv_test_488,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",488,"[{'from': 'human', 'value': 'Given a citation ..."
...,...,...,...,...,...,...,...,...,...,...
169338,3011696425,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,2020,4,arxiv_test_169338,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169338,"[{'from': 'human', 'value': 'Given a citation ..."
169339,3011708313,interpretable mtl from heterogeneous domains u...,Multi-task learning (MTL) aims at improving th...,2020,24,arxiv_test_169339,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169339,"[{'from': 'human', 'value': 'Given a citation ..."
169340,3011798063,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",2020,10,arxiv_test_169340,"{'edge_index': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10...","[{'from': 'human', 'value': 'Given a citation ...",169340,"[{'from': 'human', 'value': 'Given a citation ..."


# Splitting eval data from train data

In [22]:
eval_data = train_data[train_data["publishing_year"] == 2017]
eval_data

,paper_id,title,abstract,publishing_year,label,id,graph,conversations,node_idx,formatted_conversations
node_idx,,,,,,,,,,
268,2339293227,on design mining coevolution and surrogate models,Design mining is the use of computational inte...,2017,13,arxiv_train_268,"{'node_idx': 268, 'edge_index': [[1, 2, 3, 4, ...","[{'from': 'human', 'value': 'Given a citation ...",268,"[{'from': 'human', 'value': 'Given a citation ..."
292,2411326260,interdiction in practice hardware trojan again...,As part of the revelations about the NSA activ...,2017,4,arxiv_train_292,"{'node_idx': 292, 'edge_index': [[1, 2, 3, 4, ...","[{'from': 'human', 'value': 'Given a citation ...",292,"[{'from': 'human', 'value': 'Given a citation ..."
303,2462831000,bridging nonlinearities and stochastic regular...,We propose the Gaussian Error Linear Unit (GEL...,2017,24,arxiv_train_303,"{'node_idx': 303, 'edge_index': [[1, 2, 3, 4, ...","[{'from': 'human', 'value': 'Given a citation ...",303,"[{'from': 'human', 'value': 'Given a citation ..."
342,2574004477,distributed algorithm for collision avoidance ...,Vehicle-to-vehicle (V2V) communication is a cr...,2017,8,arxiv_train_342,"{'node_idx': 342, 'edge_index': [[1, 2, 3, 0, ...","[{'from': 'human', 'value': 'Given a citation ...",342,"[{'from': 'human', 'value': 'Given a citation ..."
343,2575436837,on the content security policy violations due ...,Modern browsers implement different security p...,2017,4,arxiv_train_343,"{'node_idx': 343, 'edge_index': [[1, 2, 3, 4, ...","[{'from': 'human', 'value': 'Given a citation ...",343,"[{'from': 'human', 'value': 'Given a citation ..."
...,...,...,...,...,...,...,...,...,...,...
169039,2952182738,partial homology relations satisfiability in t...,Directed cographs (di-cographs) play a crucial...,2017,39,arxiv_train_169039,"{'node_idx': 169039, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169039,"[{'from': 'human', 'value': 'Given a citation ..."
169067,2952694556,cooperative transmissions in ultra dense netwo...,In an Ultra-dense network (UDN) where there ar...,2017,8,arxiv_train_169067,"{'node_idx': 169067, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169067,"[{'from': 'human', 'value': 'Given a citation ..."
169089,2953039696,parallel multi channel convolution using gener...,Convolutional neural networks (CNNs) have emer...,2017,16,arxiv_train_169089,"{'node_idx': 169089, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169089,"[{'from': 'human', 'value': 'Given a citation ..."


In [23]:
train_data = train_data[train_data["publishing_year"] != 2017]
train_data

,paper_id,title,abstract,publishing_year,label,id,graph,conversations,node_idx,formatted_conversations
node_idx,,,,,,,,,,
0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",2013,4,arxiv_train_0,"{'node_idx': 0, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",0,"[{'from': 'human', 'value': 'Given a citation ..."
1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,2015,5,arxiv_train_1,"{'node_idx': 1, 'edge_index': [[1, 2, 3, 4, 0,...","[{'from': 'human', 'value': 'Given a citation ...",1,"[{'from': 'human', 'value': 'Given a citation ..."
2,116214155,on the absence of the rip in real world applic...,The purpose of this paper is twofold. The firs...,2014,28,arxiv_train_2,"{'node_idx': 2, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",2,"[{'from': 'human', 'value': 'Given a citation ..."
3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,2014,8,arxiv_train_3,"{'node_idx': 3, 'edge_index': [[1, 2, 0, 3, 4,...","[{'from': 'human', 'value': 'Given a citation ...",3,"[{'from': 'human', 'value': 'Given a citation ..."
4,231147053,analysis of asymptotically optimal sampling ba...,Over the last 20 years significant effort has ...,2014,27,arxiv_train_4,"{'node_idx': 4, 'edge_index': [[1, 2, 3, 4, 5,...","[{'from': 'human', 'value': 'Given a citation ...",4,"[{'from': 'human', 'value': 'Given a citation ..."
...,...,...,...,...,...,...,...,...,...,...
169111,2953357445,decomposition by tree dimension in horn clause...,In this paper we investigate the use of the co...,2015,2,arxiv_train_169111,"{'node_idx': 169111, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169111,"[{'from': 'human', 'value': 'Given a citation ..."
169112,2953364270,information theoretically secure three party c...,The problem in which one of three pairwise int...,2012,4,arxiv_train_169112,"{'node_idx': 169112, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169112,"[{'from': 'human', 'value': 'Given a citation ..."
169113,2953374028,network error correction for unit delay memory...,A single source network is said to be memory-f...,2009,28,arxiv_train_169113,"{'node_idx': 169113, 'edge_index': [[1, 2, 3, ...","[{'from': 'human', 'value': 'Given a citation ...",169113,"[{'from': 'human', 'value': 'Given a citation ..."


# Saving data in disk

In [24]:
!mkdir -p $PROJECT_ROOT_DIR/dataset/

In [27]:
train_data.to_parquet(f"{PROJECT_ROOT_DIR}/dataset/train.parquet")

In [28]:
eval_data.to_parquet(f"{PROJECT_ROOT_DIR}/dataset/eval.parquet")

In [29]:
test_data.to_parquet(f"{PROJECT_ROOT_DIR}/dataset/test.parquet")